### Make a copy of this note if you want to analyze 

#### Please run panta first to get gene clusters and other output files for this analysis

In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
import sys, re

In [2]:
def write_fasta(dictionary, filename):
    """
    Takes a dictionary and writes it to a fasta file
    Must specify the filename when caling the function
    https://www.programcreek.com/python/?CodeExample=write+fasta
    """

    import textwrap
    with open(filename, "w") as outfile:
        for key, value in dictionary.items():
            outfile.write(">")
            outfile.write(key + "\n")
            outfile.write("\n".join(textwrap.wrap(value, 60)))
            outfile.write("\n")

    print("Success! File written")

In [3]:
def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', '_':'_','*':'*'}
    # seq = "TCGGGCCC"
    reverse_complement = "".join(complement.get(base, base) for base in reversed(seq))
    return(reverse_complement)

In [4]:
def help_fnc(i, j):
    for ele in range(min(200,len(j)), -1, -1):
        if i.endswith(j[:ele]):
            return j[ele:]
        
def overlap(a, b):
    test_list = [a, b]
    res = ''.join(help_fnc(i, j) for i, j in zip([''] + test_list, test_list))
    return(res)

In [5]:
# overlap('AAABB', 'BBCCCC')

In [6]:
# data_dir = "../panta/examples/test/output/"
# data_dir = "data/genome_graph_test/"
# you can take the data in the directory: data/genome_graph_test
# data_dir = "../panta/examples/test/output_Kp30plus1/"
# data_dir = "../panta/examples/test/output_Kp30mix_nosplitparalog/"
# data_dir = "../panta/examples/test/output_Kp500mix/"
data_dir = "/data/hoan/amromics/simulation/art_output/spades_output/panta_output/"

In [7]:
sample_info = pd.read_csv(data_dir + "samples.tsv", delimiter='\t', header=None)
sample_info.columns = ['Name', 'SampleID']

In [8]:
sample_info.head(2)

,Name,SampleID
0,g1,0
1,ref,1


In [9]:
gene_info = pd.read_csv(data_dir + "gene_info.tsv", delimiter='\t', header=None)
gene_info.columns =['GeneName', 'SampleID', 'clusterID']

In [10]:
gene_info.head(2)

,GeneName,SampleID,clusterID
0,FHLJBAOL_00351_0_348@669@+,0,0
1,PFHEFCNI_02631_1_2563@1107@-,1,0


In [11]:
# for i in range(5000):
#     print(sum(gene_info.clusterID==0), end =",")

In [12]:
gene_position = pd.read_csv(data_dir + 'gene_position.tsv', delimiter='\t', header=None)
gene_position.columns =['SampleID', 'ContigName', 'GeneSequence']

In [13]:
gene_position.head(2)

,SampleID,ContigName,GeneSequence
0,0,NODE_1_length_360583_cov_9.851928,FHLJBAOL_00001_0_1@390@+;FHLJBAOL_00002_0_2@36...
1,0,NODE_2_length_327037_cov_9.679695,FHLJBAOL_00372_0_369@456@+;FHLJBAOL_00373_0_37...


In [14]:
# print(gene_position.ContigName.values)

In [15]:
# gene_position.iloc[20:30,]

In [16]:
# sort by length of contigs
gene_position.sort_values(by="GeneSequence", key=lambda x: x.str.len(),  ascending=False, inplace=True)

In [17]:
print("List of all samples")
n_samples = len(np.unique(gene_position.iloc[:,0]))
np.unique(gene_position.iloc[:,0])

List of all samples


array([0, 1])

In [18]:
# ## select some sample
# selected_samples = [0, 32]
# sample_info = sample_info.loc[sample_info['SampleID'].isin(selected_samples)]
# gene_info = gene_info.loc[gene_info['SampleID'].isin(selected_samples)]
# gene_position = gene_position.loc[gene_position['SampleID'].isin(selected_samples)]

# Run here

In [19]:
%load_ext autoreload
%autoreload 2
from pangraph import PanGraph

In [20]:
# construct the pangenome graph
# min_contig_len = 100
pangraph = PanGraph(sample_info=sample_info, gene_info=gene_info, gene_position=gene_position)

In [21]:
H = pangraph.construct_graph(method = "graph_alignment", sample_id_ref = None,  min_nucleotides = 100, min_genes = 0) # use the same min_contig_len when generate the ground truth
# H = pangraph.construct_graph(method = "graph_alignment", sample_id_ref = 32,  min_nucleotides = 100, min_genes = 2) # use the same min_contig_len when generate the ground truth

Set minimum on number of nucleotides =  100 NUMBER OF COMPUTED CONTIGS: 81


In [22]:
# nx.write_gml(H,'cytoscape_out/pan_graph.gml')

In [23]:
# number of nodes and edges
pangraph.n_clusters, H.number_of_edges()

(10459, 5460)

In [24]:
# pangraph.strand

In [25]:
from networkx import NetworkXNoPath

try:
    p = nx.shortest_path(H, source='C-1', target='C-0')
except NetworkXNoPath:
    p = None

In [26]:
print(2/float(len(p)))

1.0


## Data analysis

In [27]:
incomplete_sample_name = "g1"
incomplete_sample_id = sample_info[sample_info.Name==incomplete_sample_name].iloc[0,1]
contig_graph = pangraph.join_contig(sample_id=incomplete_sample_id, min_weight=1.0, method="edge_weight")
nx.write_gml(contig_graph,'cytoscape_out/contig_graph.gml')

In [28]:
incomplete_sample_id

0

In [29]:
# from matplotlib.pyplot import figure
# figure(figsize=(12, 12), dpi=80)
# nx.draw_networkx(contig_graph, arrows=True)

In [30]:
indegree_dict = dict(contig_graph.in_degree())

In [31]:
# indegree_dict

In [32]:
adj_list = {}
for source_node_key in indegree_dict:
    if indegree_dict[source_node_key] == 0:
        adj_list[source_node_key] = []
        next_neighbor_temp = source_node_key
        while(1):
            next_neighbor_temp = list(contig_graph.neighbors(next_neighbor_temp))
            if len(next_neighbor_temp) > 0:
                next_neighbor_temp = next_neighbor_temp[0]
                adj_list[source_node_key].append(next_neighbor_temp)
            else:
                break;

In [33]:
# pangraph.edge_list

In [34]:
# Read the contigs
# https://coding4medicine.com/backup/Python/reading-fasta-files.html
# f=open('/data/hoan/amromics/readSimulator/art_output/spades_output/contigs.fasta','r')
f=open('/data/hoan/amromics/simulation/art_output/spades_output/contigs.fasta','r')
lines=f.readlines()

hre=re.compile('>(\S+)')
lre=re.compile('^(\S+)$')

gene={}

for line in lines:
        outh = hre.search(line)
        if outh:
                id=outh.group(1)
        else:
                outl=lre.search(line)
                if(id in gene.keys()):
                        gene[id] += outl.group(1)
                else:
                        gene[id]  =outl.group(1)

In [35]:
# write_fasta(gene, "/data/hoan/amromics/spades_quast/SAMN04158282/contigs_test.fasta")

In [36]:
# list(gene_position.ContigName)

In [37]:
# pangraph.gene2cluster_dict['FHLJBAOL_00001_0_1@390@+']

In [38]:
from pangraph.utils import getContigsAdjacency
def append_strand(input_str):
    if input_str[-1]=="'":
        return (input_str[:-1] + '-')
    else:
        return (input_str + '+')

In [39]:
edge_list_assembly = []
# for l,r in getContigsAdjacency('/data/hoan/amromics/spades_quast/SAMN04158282/'):
for l,r in getContigsAdjacency('/data/hoan/amromics/simulation/art_output/spades_output/'):
    # print("{} --> {}".format(l,r))
    edge_list_assembly.append((append_strand(l),append_strand(r)))

In [40]:
assembly_graph = nx.DiGraph()
assembly_graph.add_edges_from(edge_list_assembly)
print("")

In [41]:
# src = "NODE_28_length_69350_cov_9.702063-"
# dst = "NODE_19_length_86979_cov_9.862615-"
# paths = [p for p in nx.all_shortest_paths(assembly_graph, src, dst)]
def flatten(l):
    if len(l) == 1:
        return l[0]
    else:
        return [item for sublist in l for item in sublist]
def remove_duplicate(your_list):
    return ([v for i, v in enumerate(your_list) if i == 0 or v != your_list[i-1]])

In [42]:
## neu ko la adjacent thi bat dau bang contigs moi.
adj_list_assembly = {}
for key in adj_list:
    new_key = key
    path1 = adj_list[key].copy()
    path1.insert(0, key)
    path2 = [new_key + pangraph.strand[new_key]]
    for i in range(len(path1)-1):
        src = path1[i] + pangraph.strand[path1[i]]
        dst = path1[i+1] + pangraph.strand[path1[i+1]]
        if not assembly_graph.has_node(src):
            path2.append(src)
        elif not assembly_graph.has_node(dst):
            path2.append(src)
            continue
        else:
            if nx.has_path(assembly_graph, src, dst):
                paths = [p for p in nx.all_shortest_paths(assembly_graph, src, dst)]
                # print(src, dst,"___", paths[0])  
                for node in paths[0]:
                    path2.append(node)
            else:
                # print(src, dst,"___", paths[0])    
                path2.append(src)
                
                print("Will test this, Ok?")
                if len(path2) > 0:
                    adj_list_assembly[new_key+pangraph.strand[new_key]] = remove_duplicate(path2)
                new_key = dst[0:-1]
                path2 = []           
    # path2 = flatten(path2)
    if dst not in path2:
        # print(path2)
        path2.append(dst)
    if len(path2) > 0:
        adj_list_assembly[new_key+pangraph.strand[new_key]] = remove_duplicate(path2)

Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?


In [43]:
# adj_list

In [44]:
# adj_list_assembly

In [45]:
gene_origin = gene.copy()
# adj_list = {}
for key in adj_list_assembly:
    path = adj_list_assembly[key]
    if len(path) > 1:
        source_node_key = key[:-1]
        if key[-1]=='-':
            gene_origin[source_node_key] = reverse_complement(gene_origin[source_node_key])

        for i in range(1, len(path)):
            # print(next_neighbor_temp)
            next_neighbor_temp = path[i][:-1]
            strand = path[i][-1]
            if strand == '+':
                gene_origin[source_node_key] = gene_origin[source_node_key]+gene[next_neighbor_temp]
                # gene_origin[source_node_key] = overlap(gene_origin[source_node_key], gene[next_neighbor_temp])
            else:
                gene_origin[source_node_key] = gene_origin[source_node_key]+reverse_complement(gene[next_neighbor_temp])
                # gene_origin[source_node_key] = overlap(gene_origin[source_node_key], reverse_complement(gene[next_neighbor_temp]))
            if next_neighbor_temp in gene_origin:
                del gene_origin[next_neighbor_temp]

In [46]:
# write_fasta(gene_origin, "/data/hoan/amromics/simulation/art_output/spades_output/contigs_concat_weight_path.fasta")
write_fasta(gene_origin, "/data/hoan/amromics/simulation/art_output/spades_output/contigs_concat.fasta")

Success! File written


In [47]:
adj_list_assembly

{'NODE_2_length_327037_cov_9.679695+': ['NODE_2_length_327037_cov_9.679695+'],
 'NODE_48_length_26212_cov_9.866692-': ['NODE_48_length_26212_cov_9.866692-'],
 'NODE_61_length_6024_cov_10.046914+': ['NODE_61_length_6024_cov_10.046914+'],
 'NODE_3_length_270336_cov_9.761447+': ['NODE_3_length_270336_cov_9.761447+'],
 'NODE_59_length_11110_cov_10.111212+': ['NODE_59_length_11110_cov_10.111212+'],
 'NODE_4_length_232574_cov_9.604416+': ['NODE_4_length_232574_cov_9.604416+',
  'NODE_70_length_1645_cov_54.421556-'],
 'NODE_56_length_15909_cov_9.153297+': ['NODE_56_length_15909_cov_9.153297+',
  'NODE_47_length_30109_cov_10.015250+',
  'NODE_28_length_69350_cov_9.702063-',
  'NODE_69_length_1723_cov_78.769745-',
  'NODE_83_length_432_cov_32.873239+',
  'NODE_92_length_245_cov_71.345238-',
  'NODE_98_length_164_cov_14.804598-',
  'NODE_64_length_2789_cov_78.536504-',
  'NODE_19_length_86979_cov_9.862615-'],
 'NODE_62_length_4411_cov_9.557914+': ['NODE_62_length_4411_cov_9.557914+'],
 'NODE_53_

In [48]:
# edge_list_assembly